In [1]:
import pandas as pd
import json
import glob
from itertools import groupby
from datetime import datetime
from IPython.core.debugger import set_trace

In [2]:
def load_all_json(limit_file=3):
    
    return [entry for d in [glob.glob("data/original_run/*.json"), glob.glob("data/second_set/*.json")]for filename in d[:limit_file] for entry in json.load(open(filename))]
     

In [23]:
RELEVANT_FIELDS = ["start_date","end_date","name","company","description","industry"]

def prepare_entry(entry):
    if entry["start_date"] and entry["end_date"]: 
        try:
            entry["start_date"] =datetime.strptime(entry["start_date"],'%Y-%m-%d').date()
            entry["end_date"] = datetime.strptime(entry["end_date"],'%Y-%m-%d').date()
            return {k:v for k,v in entry.iteritems() if k in RELEVANT_FIELDS}
        except Exception as e:
            print(e)
            return None
    else:
        return None

def prepare_data(entries):
    return [entry for entry in entries if prepare_entry(entry)]
        
def group_by_name(entries):
    entries=sorted(data, key=lambda x:x["name"])
    return groupby(entries,lambda x:x["name"])

def group_job_by_industry(jobs):
    
    sorted_jobs = sorted(jobs)
    jobs_in_industry = []
    curr_industry = [sorted_jobs[0]]
    for job in sorted_jobs[1:]:
        if job["industry"] == curr_industry[0]["industry"]:
            curr_industry.append(job)
        else:
            jobs_in_industry.append([curr_industry[0]["industry"],curr_industry])
            curr_industry =[job]
    
    jobs_in_industry.append( curr_industry[0]["industry"])
    
    return jobs_in_industry

def add_job_durations(person):
    res = []
    for job in person:
        if job["start_date"] and job["end_date"]:
            duration = (job["end_date"] - job["start_date"]).days
        else:
            duration = -1
            
        job["duration"] = duration
    
    return person

def process_persons(persons):
    
    grouped_jobs_all = {}
    for person in persons:
        name, job_list = person
        job_list = list(job_list)
        person = add_job_durations(job_list)
        grouped_jobs = group_job_by_industry(job_list)
        grouped_jobs_all[name] = grouped_jobs
        
    return grouped_jobs_all
    
    

In [24]:
data= load_all_json()


In [25]:
prepared_data = prepare_data(data)

In [26]:
grouped_data = group_by_name(prepare_data)

In [27]:
processed_persons=process_persons(grouped_data)

In [30]:
processed_persons.items()[0]

(u'James Groff', [u'Defense & Space'])